## **MACHINE LEARNING PRODUCTS (MLOps)**


***En este cuaderno jupiter se encuentra un producto minimo viable, de nuestros proyecto - "Sistema de Recomendacion del Mundial del 2026" solo vamos  a utilizar solo la data de Google Maps***

In [1]:
# Librerias
import pymysql
import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine
from dotenv import load_dotenv
import pandas as pd
import os

In [3]:
# La data se encuentra en AWS vamos a conectar
# Cargamos el archivo .env
load_dotenv()

# Acceder a las variables entorno
password_bd = os.getenv("PASS_BD")
user_bd = os.getenv("USER_BD")
host_name = os.getenv("HOST_NAME")
name_bd = os.getenv("NAME_BD")

# Conectamos la base de datos


def BD_connection(host_name, user_bd, user_pass, name_bd):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_bd,
            passwd=user_pass,
            database=name_bd,
            port=3306
        )
        print("MYSQL DATABASE connection succesful")
    except Error as err:
        print(f"Error: '{err}'")
    return connection

In [29]:
connection = BD_connection(host_name, user_bd, password_bd, name_bd)

MYSQL DATABASE connection succesful


In [12]:
# verifiquemos la conexion realizando una consulta simple
if connection:
    cursor = connection.cursor()
    cursor.execute("SELECT DATABASE()")
    db = cursor.fetchone()
    print(f"Conectado a la base de datos: {db[0]}")

Conectado a la base de datos: Google


In [13]:
# Funcion para obtener las tablas disponibles
def show_tables(connection):
    cursor = connection.cursor()
    cursor.execute("SHOW TABLES")
    tables = cursor.fetchall()
    for table in tables:
        print(table)

In [14]:
# Corremos la query
connection = BD_connection(host_name, user_bd, password_bd, name_bd)
show_tables(connection)

MYSQL DATABASE connection succesful
('sites',)
('users_google',)


In [20]:
# Funcion para describir la estructura de una tabla 
def describe_table(connection, table_name):
    cursor = connection.cursor()
    query = f"DESCRIBE {table_name}"
    cursor.execute(query)
    columns = cursor.fetchall()
    for column in columns:
        print(column)

In [21]:
describe_table(connection, 'users_google')

('id_user', 'varchar(100)', 'NO', 'PRI', None, '')
('name', 'varchar(50)', 'NO', '', None, '')
('time_comment', 'date', 'YES', '', None, '')
('text_comment', 'varchar(1000)', 'YES', '', None, '')
('raiting', 'int', 'YES', '', None, '')
('pics', 'varchar(2048)', 'YES', '', None, '')
('time_resp', 'date', 'YES', '', None, '')
('text_resp', 'varchar(1000)', 'YES', '', None, '')
('gmap_id', 'varchar(100)', 'YES', 'MUL', None, '')
('state', 'varchar(30)', 'YES', '', None, '')


In [30]:
# Vamos a utilizar esta funcion para correr queries
def execute_query(connection, query):
    cursor = connection.cursor()
    try: 
        cursor.execute(query)
        connection.commit()
        print("Query hecha con éxito")
    except Error as err:
        print("Error", err)

In [31]:
execute_query(connection, "SELECT * FROM users_google")

Error Unread result found
